# Implementation of the language models

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.text.models import * 
from fastai import *

This module fully implements the [AWD-LSTM](https://arxiv.org/pdf/1708.02182.pdf) from Stephen Merity et al. The main idea of the article is to use a [RNN](http://www.pnas.org/content/79/8/2554) with dropout everywhere, but in an intelligent way. There is a difference with the usual dropout, which is why you’ll see a [`RNNDropout`](/text.models.html#RNNDropout) module: we zero things, as is usual in dropout, but we always zero the same thing according to the sequence dimension (which is the first dimension in pytorch). This ensures consistency when updating the hidden state through the whole sentences/articles. 

This being given, there are five different dropouts in the AWD-LSTM:
- the first one, embedding dropout, is applied when we look the ids of our tokens inside the embedding matrix (to transform them from numbers to a vector of float). We zero some lines of it, so random ids are sent to a vector of zeros instead of being sent to their embedding vector.
- the second one, input dropout, is applied to the result of the embedding with dropout. We forget random pieces of the embedding matrix (but as stated in the last paragraph, the same ones in the sequence dimension).
- the third one is the weight dropout. It’s the trickiest to implement as we randomly replace by 0s some weights of the hidden-to-hidden matrix inside the RNN: this needs to be done in a way that ensure the gradients are still computed and the initial weights still updated.
- the fourth one is the hidden dropout. It’s applied to the output of one of the layers of the RNN before it’s used as input of the next layer (again same coordinates are zeroed in the sequence dimension). This one isn’t applied to the last output, but rather…
- the fifth one is the output dropout, it’s applied to the last output of the model (and like the others, it’s applied the same way through the first dimension).

## Basic functions to get a model

In [ ]:
show_doc(get_language_model, doc_string=False)

#### <a id=get_language_model></a>`get_language_model`
> `get_language_model`(`vocab_sz`:`int`, `emb_sz`:`int`, `n_hid`:`int`, `n_layers`:`int`, `pad_token`:`int`, `tie_weights`:`bool`=`True`, `qrnn`:`bool`=`False`, `bias`:`bool`=`True`, `bidir`:`bool`=`False`, `output_p`:`float`=`0.4`, `hidden_p`:`float`=`0.2`, `input_p`:`float`=`0.6`, `embed_p`:`float`=`0.1`, `weight_p`:`float`=`0.5`) -> [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L205">[source]</a>

#### <a id=get_language_model></a>`get_language_model`
> `get_language_model`(`vocab_sz`:`int`, `emb_sz`:`int`, `n_hid`:`int`, `n_layers`:`int`, `pad_token`:`int`, `tie_weights`:`bool`=`True`, `qrnn`:`bool`=`False`, `bias`:`bool`=`True`, `bidir`:`bool`=`False`, `output_p`:`float`=`0.4`, `hidden_p`:`float`=`0.2`, `input_p`:`float`=`0.6`, `embed_p`:`float`=`0.1`, `weight_p`:`float`=`0.5`) -> [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L205">[source]</a>

Creates an AWD-LSTM with a first embedding of `vocab_sz` by `emb_sz`, a hidden size of `n_hid`, RNNs with `n_layers` that can be bidirectional if `bidir` is True. The last RNN as an output size of `emb_sz` so that we can use the same decoder as the encoder if `tie_weights` is True. The decoder is a `Linear` layer with or without `bias`. If `qrnn` is set to True, we use [QRNN cells] instead of LSTMS. `pad_token` is the token used for padding.

`embed_p` is used for the embedding dropout, `input_p` is used for the input dropout, `weight_p` is used for the weight dropout, `hidden_p` is used for the hidden dropout and `output_p` is used for the output dropout.

Note that the model returns a list of three things, the actual output being the first, the two others being the intermediate hidden states before and after dropout (used by the [`RNNTrainer`](/callbacks.rnn.html#RNNTrainer)). Most loss functions expect one output, so you should use a Callback to remove the other two if you're not using [`RNNTrainer`](/callbacks.rnn.html#RNNTrainer).

In [ ]:
show_doc(get_rnn_classifier, doc_string=False)

#### <a id=get_rnn_classifier></a>`get_rnn_classifier`
> `get_rnn_classifier`(`bptt`:`int`, `max_seq`:`int`, `n_class`:`int`, `vocab_sz`:`int`, `emb_sz`:`int`, `n_hid`:`int`, `n_layers`:`int`, `pad_token`:`int`, `layers`:`Collection`\[`int`\], `drops`:`Collection`\[`float`\], `bidir`:`bool`=`False`, `qrnn`:`bool`=`False`, `hidden_p`:`float`=`0.2`, `input_p`:`float`=`0.6`, `embed_p`:`float`=`0.1`, `weight_p`:`float`=`0.5`) -> [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L214">[source]</a>

#### <a id=get_rnn_classifier></a>`get_rnn_classifier`
> `get_rnn_classifier`(`bptt`:`int`, `max_seq`:`int`, `n_class`:`int`, `vocab_sz`:`int`, `emb_sz`:`int`, `n_hid`:`int`, `n_layers`:`int`, `pad_token`:`int`, `layers`:`Collection`\[`int`\], `drops`:`Collection`\[`float`\], `bidir`:`bool`=`False`, `qrnn`:`bool`=`False`, `hidden_p`:`float`=`0.2`, `input_p`:`float`=`0.6`, `embed_p`:`float`=`0.1`, `weight_p`:`float`=`0.5`) -> [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L214">[source]</a>

Creates a RNN classifier with a encoder taken from an AWD-LSTM with arguments `vocab_sz`, `emb_sz`, `n_hid`, `n_layers`, `bias`, `bidir`, `qrnn`, `pad_token` and the dropouts parameters. This encoder is fed the sequence by successive bits of size `bptt` and we only keep the last `max_seq` outputs for the pooling layers.

The decoder use a concatenation of the last outputs, a `MaxPooling` of all the ouputs and an `AveragePooling` of all the outputs. It then uses a list of `BatchNorm`, `Dropout`, `Linear`, `ReLU` blocks (with no `ReLU` in the last one), using a first layer size of `3*emb_sz` then follwoing the numbers in `n_layers` to stop at `n_class`. The dropouts probabilities are read in `drops`.

Note that the model returns a list of three things, the actual output being the first, the two others being the intermediate hidden states before and after dropout (used by the [`RNNTrainer`](/callbacks.rnn.html#RNNTrainer)). Most loss functions expect one output, so you should use a Callback to remove the other two if you're not using [`RNNTrainer`](/callbacks.rnn.html#RNNTrainer).

## Basic NLP modules

On top of the pytorch or the fastai [`layers`](/layers.html#layers), the language models use some custom layers specific to NLP.

In [ ]:
show_doc(EmbeddingDropout, doc_string=False, title_level=3)

### <a id=EmbeddingDropout></a>`class` `EmbeddingDropout`
> `EmbeddingDropout`(`emb`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `embed_p`:`float`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L50">[source]</a>

### <a id=EmbeddingDropout></a>`class` `EmbeddingDropout`
> `EmbeddingDropout`(`emb`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `embed_p`:`float`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L50">[source]</a>

Applies a dropout with probability `embed_p` to an embedding layer `emb` in training mode. Each row of the embedding matrix has a probability `embed_p` of being replaced by zeros while the others are rescaled accordingly. 

In [ ]:
enc = nn.Embedding(100, 7, padding_idx=1)
enc_dp = EmbeddingDropout(enc, 0.5)
tst_input = torch.randint(0,100,(8,))
enc_dp(tst_input)

tensor([[ 1.6726,  0.0368,  1.4787,  3.4628,  1.0888,  3.0076, -2.0304],
        [ 0.0000,  0.0000,  0.0000, -0.0000, -0.0000,  0.0000, -0.0000],
        [ 0.1830, -1.3432,  1.1055,  0.4599,  3.7144, -3.1382,  0.0172],
        [ 1.8115,  2.6855,  2.7093, -0.2569,  0.9212, -2.2593,  3.8310],
        [-0.0000, -0.0000, -0.0000,  0.0000, -0.0000, -0.0000,  0.0000],
        [-0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -0.0000,  0.0000],
        [-0.2968,  2.1759,  1.0536, -3.4618, -2.2319, -1.5875,  1.4099],
        [-0.0000,  0.0000, -0.0000, -0.0000, -0.0000, -0.0000,  0.0000]],
       grad_fn=<EmbeddingBackward>)

In [ ]:
show_doc(RNNDropout, doc_string=False, title_level=3)

### <a id=RNNDropout></a>`class` `RNNDropout`
> `RNNDropout`(`p`:`float`=`0.5`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L11">[source]</a>

### <a id=RNNDropout></a>`class` `RNNDropout`
> `RNNDropout`(`p`:`float`=`0.5`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L11">[source]</a>

Applies a dropout with probability `p` consistently over the first dimension in training mode.

In [ ]:
dp = RNNDropout(0.3)
tst_input = torch.randn(3,3,7)
tst_input, dp(tst_input)

(tensor([[[ 0.8305, -2.6687,  0.1399,  0.8773, -0.9601, -1.9875, -0.7754],
          [-0.8539,  0.4316,  1.9667,  1.3948,  0.8318, -1.6018,  1.6207],
          [-0.0294,  1.4592, -0.8035, -0.2747, -0.8405,  0.4735, -1.4664]],
 
         [[-0.8362, -0.2606,  3.2887,  0.0123,  0.2857,  0.0662, -0.8583],
          [-0.0081, -0.9123, -0.3900, -0.7823,  1.7898,  0.9905,  0.6019],
          [ 0.5715,  2.0270,  0.5443,  1.4103,  0.0209, -1.2136,  0.0393]],
 
         [[ 1.1452, -0.9102,  0.6555, -0.6423,  1.2612,  1.9358, -0.1744],
          [ 1.2404,  2.1373, -1.1495, -0.9078,  0.8448, -0.6073, -0.2978],
          [-0.6869, -0.8378,  0.2918, -0.4366,  0.4442,  1.7649,  1.2987]]]),
 tensor([[[ 1.1865, -0.0000,  0.1998,  1.2533, -1.3716, -0.0000, -0.0000],
          [-0.0000,  0.6166,  2.8096,  0.0000,  0.0000, -2.2883,  0.0000],
          [-0.0000,  2.0846, -1.1479, -0.3924, -1.2008,  0.0000, -2.0948]],
 
         [[-1.1945, -0.0000,  4.6981,  0.0176,  0.4082,  0.0000, -0.0000],
          [-0

In [ ]:
show_doc(WeightDropout, doc_string=False, title_level=3)

### <a id=WeightDropout></a>`class` `WeightDropout`
> `WeightDropout`(`module`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `weight_p`:`float`, `layer_names`:`StrList`=`['weight_hh_l0']`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L23">[source]</a>

### <a id=WeightDropout></a>`class` `WeightDropout`
> `WeightDropout`(`module`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `weight_p`:`float`, `layer_names`:`StrList`=`['weight_hh_l0']`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L23">[source]</a>

Applies dropout of probability `weight_p` to the layers in `layer_names` of `module` in training mode. A copy of those weights is kept so that the dropout mask can change at every batch.

In [ ]:
module = nn.LSTM(5, 2)
dp_module = WeightDropout(module, 0.4)
getattr(dp_module.module, 'weight_hh_l0')

Parameter containing:
tensor([[-0.5222,  0.4933],
        [ 0.5108, -0.3863],
        [ 0.4076, -0.4272],
        [ 0.3383, -0.5260],
        [-0.0579,  0.4174],
        [ 0.5398,  0.6208],
        [-0.4088,  0.2255],
        [-0.5841, -0.3922]], requires_grad=True)

It's at the beginning of a forward pass that the dropout is applied to the weights.

In [ ]:
tst_input = torch.randn(4,20,5)
h = (torch.zeros(1,20,2), torch.zeros(1,20,2))
x,h = dp_module(tst_input,h)
getattr(dp_module.module, 'weight_hh_l0')

tensor([[-0.8704,  0.0000],
        [ 0.0000, -0.0000],
        [ 0.6794, -0.7120],
        [ 0.5639, -0.8767],
        [-0.0000,  0.6956],
        [ 0.8997,  1.0347],
        [-0.0000,  0.3759],
        [-0.9734, -0.0000]], grad_fn=<MulBackward0>)

In [ ]:
show_doc(SequentialRNN, doc_string=False, title_level=3)

### <a id=SequentialRNN></a>`class` `SequentialRNN`
> `SequentialRNN`(`args`) :: [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L151">[source]</a>

### <a id=SequentialRNN></a>`class` `SequentialRNN`
> `SequentialRNN`(`args`) :: [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L151">[source]</a>

Create a `Sequentiall` module with `args` that has a `reset` function.

In [ ]:
show_doc(SequentialRNN.reset)

#### <a id=reset></a>`reset`
> `reset`()
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L153">[source]</a>

#### <a id=reset></a>`reset`
> `reset`()
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L153">[source]</a>

Call the `reset` function of [`self.children`](/torch_core.html#children) (if they have one).

In [ ]:
show_doc(dropout_mask, doc_string=False)

#### <a id=dropout_mask></a>`dropout_mask`
> `dropout_mask`(`x`:`Tensor`, `sz`:`Collection`\[`int`\], `p`:`float`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L7">[source]</a>

#### <a id=dropout_mask></a>`dropout_mask`
> `dropout_mask`(`x`:`Tensor`, `sz`:`Collection`\[`int`\], `p`:`float`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L7">[source]</a>

Create a dropout mask of size `sz`, the same type as `x` and probability `p`.

In [ ]:
tst_input = torch.randn(3,3,7)
dropout_mask(tst_input, (3,7), 0.3)

tensor([[1.4286, 1.4286, 1.4286, 1.4286, 0.0000, 1.4286, 0.0000],
        [1.4286, 0.0000, 1.4286, 1.4286, 1.4286, 1.4286, 1.4286],
        [1.4286, 1.4286, 1.4286, 1.4286, 1.4286, 1.4286, 1.4286]])

Such a mask is then expanded in the sequence length dimension and multiplied by the input to do an [`RNNDropout`](/text.models.html#RNNDropout).

## Language model modules

In [ ]:
show_doc(RNNCore, doc_string=False, title_level=3)

### <a id=RNNCore></a>`class` `RNNCore`
> `RNNCore`(`vocab_sz`:`int`, `emb_sz`:`int`, `n_hid`:`int`, `n_layers`:`int`, `pad_token`:`int`, `bidir`:`bool`=`False`, `hidden_p`:`float`=`0.2`, `input_p`:`float`=`0.6`, `embed_p`:`float`=`0.1`, `weight_p`:`float`=`0.5`, `qrnn`:`bool`=`False`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L73">[source]</a>

### <a id=RNNCore></a>`class` `RNNCore`
> `RNNCore`(`vocab_sz`:`int`, `emb_sz`:`int`, `n_hid`:`int`, `n_layers`:`int`, `pad_token`:`int`, `bidir`:`bool`=`False`, `hidden_p`:`float`=`0.2`, `input_p`:`float`=`0.6`, `embed_p`:`float`=`0.1`, `weight_p`:`float`=`0.5`, `qrnn`:`bool`=`False`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L73">[source]</a>

Create an AWD-LSTM encoder with an embedding layer of `vocab_sz` by `emb_sz`, a hidden size of `n_hid`, `n_layers` layers. `pad_token` is passed to the `Embedding`, if `bidir` is True, the model is bidirectional. If `qrnn` is True, we use QRNN cells instead of LSTMs. Dropouts are `embed_p`, `input_p`, `weight_p` and `hidden_p`.

In [ ]:
show_doc(RNNCore.reset)

#### <a id=reset></a>`reset`
> `reset`()


Reset the hidden states. <a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L125">[source]</a>

#### <a id=reset></a>`reset`
> `reset`()


Reset the hidden states. <a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L125">[source]</a>

In [ ]:
show_doc(LinearDecoder, doc_string=False, title_level=3)

### <a id=LinearDecoder></a>`class` `LinearDecoder`
> `LinearDecoder`(`n_out`:`int`, `n_hid`:`int`, `output_p`:`float`, `tie_encoder`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)=`None`, `bias`:`bool`=`True`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L132">[source]</a>

### <a id=LinearDecoder></a>`class` `LinearDecoder`
> `LinearDecoder`(`n_out`:`int`, `n_hid`:`int`, `output_p`:`float`, `tie_encoder`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)=`None`, `bias`:`bool`=`True`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L132">[source]</a>

Create a the decoder to go on top of an [`RNNCore`](/text.models.html#RNNCore) encoder and create a language model. `n_hid` is the dimension of the last hidden state of the encoder, `n_out` the size of the output. Dropout of `output_p` is applied. If a `tie_encoder` is passed, it will be used for the weights of the linear layer, that will have `bias` or not.

## Classifier modules

In [ ]:
show_doc(MultiBatchRNNCore, doc_string=False, title_level=3)

### <a id=MultiBatchRNNCore></a>`class` `MultiBatchRNNCore`
> `MultiBatchRNNCore`(`bptt`:`int`, `max_seq`:`int`, `args`, `kwargs`) :: [`RNNCore`](/text.models.html#RNNCore)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L157">[source]</a>

### <a id=MultiBatchRNNCore></a>`class` `MultiBatchRNNCore`
> `MultiBatchRNNCore`(`bptt`:`int`, `max_seq`:`int`, `args`, `kwargs`) :: [`RNNCore`](/text.models.html#RNNCore)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L157">[source]</a>

Wrap an [`RNNCore`](/text.models.html#RNNCore) to make it process full sentences: text is passed by chunks of sequence length `bptt` and only the last `max_seq` outputs are kept for the next layer. `args` and `kwargs` are passed to the [`RNNCore`](/text.models.html#RNNCore).

In [ ]:
show_doc(MultiBatchRNNCore.concat)

#### <a id=concat></a>`concat`
> `concat`(`arrs`:`Collection`\[`Tensor`\]) -> `Tensor`


Concatenate the `arrs` along the batch dimension. <a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L164">[source]</a>

#### <a id=concat></a>`concat`
> `concat`(`arrs`:`Collection`\[`Tensor`\]) -> `Tensor`


Concatenate the `arrs` along the batch dimension. <a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L164">[source]</a>

In [ ]:
show_doc(PoolingLinearClassifier, doc_string=False, title_level=3)

### <a id=PoolingLinearClassifier></a>`class` `PoolingLinearClassifier`
> `PoolingLinearClassifier`(`layers`:`Collection`\[`int`\], `drops`:`Collection`\[`float`\]) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L179">[source]</a>

### <a id=PoolingLinearClassifier></a>`class` `PoolingLinearClassifier`
> `PoolingLinearClassifier`(`layers`:`Collection`\[`int`\], `drops`:`Collection`\[`float`\]) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L179">[source]</a>

Create a linear classifier that sits on an [`RNNCore`](/text.models.html#RNNCore) encoder. The last output, `MaxPooling` of all the outputs and `AvgPooling` of all the outputs are concatenated, then blocks of [`bn_drop_lin`](/layers.html#bn_drop_lin) are stacked, according to the values in [`layers`](/layers.html#layers) and `drops`.

In [ ]:
show_doc(PoolingLinearClassifier.pool, doc_string=False)

#### <a id=pool></a>`pool`
> `pool`(`x`:`Tensor`, `bs`:`int`, `is_max`:`bool`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L190">[source]</a>

#### <a id=pool></a>`pool`
> `pool`(`x`:`Tensor`, `bs`:`int`, `is_max`:`bool`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L190">[source]</a>

Pool `x` (of batch size `bs`) along the batch dimension. `is_max` decides if we do an `AvgPooling` or a `MaxPooling`.

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(WeightDropout.forward)

#### <a id=forward></a>`forward`
> `forward`(`args`:`Classes`)


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. <a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L40">[source]</a>

#### <a id=forward></a>`forward`
> `forward`(`args`:`Classes`)


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. <a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L40">[source]</a>

In [ ]:
show_doc(RNNCore.forward)

#### <a id=forward></a>`forward`
> `forward`(`input`:`LongTensor`) -> `Tuple`\[`Tensor`, `Tensor`\]


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. <a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L104">[source]</a>

#### <a id=forward></a>`forward`
> `forward`(`input`:`LongTensor`) -> `Tuple`\[`Tensor`, `Tensor`\]


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. <a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L104">[source]</a>

In [ ]:
show_doc(EmbeddingDropout.forward)

#### <a id=forward></a>`forward`
> `forward`(`words`:`LongTensor`, `scale`:`Optional`\[`float`\]=`None`) -> `Tensor`


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. <a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L59">[source]</a>

#### <a id=forward></a>`forward`
> `forward`(`words`:`LongTensor`, `scale`:`Optional`\[`float`\]=`None`) -> `Tensor`


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. <a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L59">[source]</a>

In [ ]:
show_doc(RNNDropout.forward)

#### <a id=forward></a>`forward`
> `forward`(`x`:`Tensor`) -> `Tensor`


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. <a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L18">[source]</a>

#### <a id=forward></a>`forward`
> `forward`(`x`:`Tensor`) -> `Tensor`


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. <a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L18">[source]</a>

## New Methods - Please document or move to the undocumented section